In [ ]:
import astropy.units as u
from astropy.constants import c
from astropy.io import ascii
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
data = ascii.read('../data/RXJ1713_HESS_2007.dat')

# SIMPLE TASKS

In [ ]:
from naima.models import ExponentialCutoffPowerLaw, PionDecay

In [ ]:
def Model(pars, Es):
    """
    Define particle distribution model, radiative model, and return model flux
    at energy values
    """

    ECPL = ExponentialCutoffPowerLaw(
        pars[0] / u.eV, 10.0 * u.TeV, pars[1], 10 ** pars[2] * u.TeV
    )
    PD = PionDecay(ECPL, nh = pars[3] / u.cm**3, Epmax = 10 ** pars[4] * u.PeV)
    
    Wp = PD.compute_Wp()
    
    Ep = np.logspace(np.log10(PD.Epmin.to(u.GeV).value), 
                     np.log10(PD.Epmax.to(u.GeV).value),
                     100
                    ) * u.eV

    return PD.flux(Es, distance=1.0*u.kpc), (Ep, ECPL(Ep)), Wp

In [ ]:
## Set initial parameters and labels
# (Use log energy fir fitting purposes later)
labels = ['amplitude_eV', 'alpha', 'log_cutoff_energy_TeV', 'nH_cm3', 'log_Epmax_PeV']
p0 = np.array((2e33, 2.0, np.log10(150), 50, np.log10(1)))

### TASK: Take base model and change iteratively each parameter to investigate their influence on the model

In [ ]:
from naima.models import ### ???

In [ ]:
# Plot data
mask = data['ul'] == 0
plt.plot((data['energy'][mask]).to(u.eV), data['energy'][mask].to(u.erg)**2 * (data['flux'][mask]).to(1 / u.erg / u.cm**2 / u.s), 'o')

# Base model
Egs = np.logspace(5, 15, 100) * u.eV
p0 = np.array((2e33, 2.0, np.log10(150), 50, np.log10(1)))
plt.plot(Egs.to(u.eV), (Egs**2 * Model(p0, Egs)[0]).to(u.erg / u.cm**2 / u.s))

# Comparative model
p0 = np.array((### MAKE CHANGES HERE####))
plt.plot(Egs.to(u.eV), (Egs**2 * Model(p0, Egs)[0]).to(u.erg / u.cm**2 / u.s))

plt.xscale('log')
plt.yscale('log')

plt.xlabel(f'Photon Energy / eV')
plt.ylabel(f'$E^2$ d$N$/d$E$ / (eV / (cm2 s))')
    
plt.ylim(1e-13,1e-10);

### TASK: What’s the total proton kinetic energy? Is it realistic for such as source?

In [ ]:
### WRITE CODE HERE ###

### TASK: Implement new functions with alternative models, for example using a different distribution of the parent population and / or a different radiative model. Determine suitable model parameters by trial and error.

In [ ]:
def Model2(pars, Es):
    """
    Define particle distribution model, radiative model, and return model flux
    at energy values
    """

    ## CODE HERE PARENT DISTRIBUTION
    
    ## CODE HERE RADIATIVE MODEL
    
    return ##??##.flux(Es, distance=1.0*u.kpc)

In [ ]:
# Plot data
mask = data['ul'] == 0
plt.plot((data['energy'][mask]).to(u.eV), data['energy'][mask].to(u.erg)**2 * (data['flux'][mask]).to(1 / u.erg / u.cm**2 / u.s), 'o')

# Base model
Egs = np.logspace(5, 15, 100) * u.eV
labels = [### ?? ###]
p0 = np.array((### ?? ###)
plt.plot(Egs.to(u.eV), (Egs**2 * Model2(p0, Egs)[0]).to(u.erg / u.cm**2 / u.s))

plt.xlabel(f'Photon Energy / eV')
plt.ylabel(f'$E^2$ d$N$/d$E$ / (eV / (cm2 s))')

plt.xscale('log')
plt.yscale('log')

plt.ylim(1e-13,1e-10);

# Advanced Tasks

### TASK: Take one of the above models and perform a MCMC fit.
Tips (to reduce comp. time):
- Determine rough parameter ranges for the priors by varying parameters by hand
- Reduce number of fit parameters (e.g. if the influence of a parameter is marginal, fix the parameter; if two parameters influence spectrum the same, only fit one of them
Keep nwalkers, nburn, nrun small (e.g. 10, 50, 10)

In [ ]:
from naima import uniform_prior
from naima import run_sampler

In [ ]:
labels = [### ?? ###]
p0 = np.array(### ?? ###)

In [ ]:
def lnprior(pars):
    logprob = uniform_prior(pars[0], #?#, #?#) \
            + uniform_prior(pars[1], #?#, #?#) \
            ## ?? ##
    return logprob

In [ ]:
## Run sampler
sampler, pos = run_sampler(
        data_table=data,
        p0=p0,
        labels=labels,
        model=Model,
        prior=lnprior,
        nwalkers=10,
        nburn=50,
        nrun=10,
        threads=4,
        prefit=True,
        interactive=False,
    )

In [ ]:
from naima import plot_corner, plot_data, plot_chain, plot_fit

In [ ]:
plot_corner(sampler);

In [ ]:
plot_fit(sampler);

In [ ]:
plot_chain(sampler)